In [1]:
from bcgpeaks import findpeaks_parallel
from bcg import load_bcg

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
bcg_path = "/home/guo/BCGHeart/data/wsx_bcg_wave(500HZ).txt"
signal = load_bcg(bcg_path, offset=12726)

In [4]:
from plotly.offline import plot
import plotly.graph_objs as go

In [5]:
plot([go.Scatter(y=signal)])

'file:///home/guo/Github/BCGHeart/bcgpeaks/temp-plot.html'

In [6]:
segment = signal[400000:410000]

In [7]:
plot([go.Scatter(y=segment)])

'file:///home/guo/Github/BCGHeart/bcgpeaks/temp-plot.html'

In [8]:
plot([go.Scatter(y=segment[5742:6523]/2.296)])

'file:///home/guo/Github/BCGHeart/bcgpeaks/temp-plot.html'

In [9]:
np.max(segment) + np.abs(np.min(segment))

1667.1619986969922

In [10]:
np.max(segment[:5700]) + np.abs(np.min(segment[:5700]))

726.2996169860317

In [11]:
1667/726

2.296143250688705

In [12]:
new_segement = np.copy(segment)

In [13]:
new_segement[5742:6523] = segment[5742:6523]/2.296

In [14]:
plot([go.Scatter(y=segment), go.Scatter(y=new_segement)])

'file:///home/guo/Github/BCGHeart/bcgpeaks/temp-plot.html'

# load bcg and  ecg

In [24]:
from bcg import load_bcg
from edfplus import Edfplus
import autopeaks

In [25]:
bcg = load_bcg("/home/guo/BCGHeart/data/wsx_bcg_wave(500HZ).txt", offset=12726, filter=True)

In [26]:
f = open('../data/wsx_ecg.edf', 'rb')
raw = f.read()
f.close()

In [27]:
edf = Edfplus(raw)
ecg = edf.signals['ECG LL-RA']

In [28]:
ecg_auto_peaks = autopeaks.AutoPeaks(thres=0.75,min_dist=300,fs=500)

In [29]:
list(map(ecg_auto_peaks.findpeaks, ecg));

In [30]:
ecg_peak_indices = ecg_auto_peaks.peak_indexes
ecg_peak_values = ecg_auto_peaks.peak_values

In [31]:
plot([go.Scatter(y=ecg), go.Scatter(x=ecg_peak_indices, y=ecg_peak_values, mode="markers")])

'file:///home/guo/Github/BCGHeart/bcgpeaks/temp-plot.html'

# detect body movement

In [32]:
import autopeaks

In [33]:
bcg_auto_peaks = autopeaks.AutoPeaks(thres=0.50, min_dist=60, fs=500,buffer_size=200)

In [34]:
list(map(bcg_auto_peaks.findpeaks, signal));

In [35]:
bcg_peak_indices = bcg_auto_peaks.peak_indexes
bcg_peak_values = bcg_auto_peaks.peak_values

In [36]:
plot([go.Scatter(y=bcg), go.Scatter(x=bcg_peak_indices, y=bcg_peak_values, mode="markers")])

'file:///home/guo/Github/BCGHeart/bcgpeaks/temp-plot.html'

### output peak baseline for body movement

In [37]:
from collections import deque

In [38]:
base_buffer = deque(maxlen=60*5)

In [39]:
bases = []
base_buffer.append(bcg_peak_values[0])
for value in bcg_peak_values:
    l = list((base_buffer))
    l.sort()
    base = l[int(0.75*len(l))]
    bases.append(base)
    base_buffer.append(value)

In [40]:
plot([go.Scatter(y=bcg), go.Scatter(x=bcg_peak_indices, y=bcg_peak_values, mode="markers"), go.Scatter(x=bcg_peak_indices,y=bases,mode="lines")])

'file:///home/guo/Github/BCGHeart/bcgpeaks/temp-plot.html'

# body movement detection

In [41]:
import bcgpeaks

In [42]:
from bcgpeaks import body_movement

In [43]:
mts = body_movement(bcg)

In [44]:
shapes = [{'type': 'rect', 'xref': 'x', 'yref': 'y', 'x0': start, 'y0': bcg[start]*5, 'x1': end,'y1': bcg[end]*(-5), 'opacity': 0.4, 'fillcolor': 'green', 'line': {
                'color': 'green',
            },
        } for (start,end) in mts]

In [45]:
layout = {'shapes':shapes}

In [46]:
plot({"data":[go.Scatter(y=bcg), go.Scatter(x=bcg_peak_indices, y=bcg_peak_values, mode="markers"), go.Scatter(x=bcg_peak_indices,y=bases,mode="lines")],"layout":layout},)

'file:///home/guo/Github/BCGHeart/bcgpeaks/temp-plot.html'

# repair MT to normal signal

In [83]:
from bcgpeaks import repair_body_movement

In [85]:
new_bcg = repair_body_movement(bcg, mts);

In [87]:
offset = 500*60*1000

In [86]:
plot([go.Scatter(y=new_bcg[:offset], name="new bcg"), go.Scatter(y=bcg[:offset]+4000,name="bcg")])

'file:///home/guo/Github/BCGHeart/bcgpeaks/temp-plot.html'

In [81]:
plot([go.Scatter(y=old,name="old"),go.Scatter(y=new_bcg,name="new")])

'file:///home/guo/Github/BCGHeart/bcgpeaks/temp-plot.html'

In [ ]:
len(bcg)

In [ ]:
len(mts)

In [ ]:
len(new_bcg)

In [51]:
plot([go.Scatter(y=new_bcg[:offset], name="new bcg")])

'file:///home/guo/Github/BCGHeart/bcgpeaks/temp-plot.html'

In [95]:
bcg_auto_peaks = autopeaks.AutoPeaks(thres=0.70,min_dist=300,fs=500)

In [96]:
list(map(bcg_auto_peaks.findpeaks, new_bcg));

In [97]:
bcg_peak_indices = bcg_auto_peaks.peak_indexes
bcg_peak_values = bcg_auto_peaks.peak_values

In [98]:
plot([go.Scatter(y=ecg),go.Scatter(y=bcg), go.Scatter(x=bcg_peak_indices, y=bcg_peak_values, mode="markers")])

'file:///home/guo/Github/BCGHeart/bcgpeaks/temp-plot.html'

In [99]:
ecg_rr = np.diff(ecg_peak_indices)

In [100]:
bcg_jj = np.diff(bcg_peak_indices)

In [107]:
plot([go.Scatter(x=ecg_peak_indices[1:],y=ecg_rr, mode="lines+markers",name="ecg"), go.Scatter(x=np.array(bcg_peak_indices[1:]),y=bcg_jj,mode="lines+markers", name="bcg")])

'file:///home/guo/Github/BCGHeart/bcgpeaks/temp-plot.html'

In [102]:
from scipy.signal import medfilt

In [113]:
med_jj = medfilt(bcg_jj, kernel_size=7)

In [114]:
plot([go.Scatter(x=bcg_peak_indices[1:], y=bcg_jj, name="bcg_JJ", mode="lines+markers"),go.Scatter(x=bcg_peak_indices[1:], y=med_jj, name="med_JJ",mode="lines+markers"),go.Scatter(x=ecg_peak_indices[1:], y=ecg_rr, name="ecg_rr",mode="lines+markers")])

'file:///home/guo/Github/BCGHeart/bcgpeaks/temp-plot.html'

In [115]:
plot([go.Scatter(x=bcg_peak_indices[1:], y=med_jj, name="med_JJ",mode="lines+markers"),go.Scatter(x=ecg_peak_indices[1:], y=ecg_rr, name="ecg_rr",mode="lines+markers")])

'file:///home/guo/Github/BCGHeart/bcgpeaks/temp-plot.html'

# HMW ecg analysis

In [118]:
hwm_ecg_path = "/home/guo/BCGHeart/data/bcg_ecg_data(500HZ)/hwm_ecg.edf"

In [119]:
def load_ecg(filepath):
    f = open(filepath, 'rb')
    raw = f.read()
    f.close()
    edf = Edfplus(raw)
    return edf.signals["ECG LL-RA"]

In [120]:
hwm_ecg = load_ecg(hwm_ecg_path)

In [142]:
def ecg_findpeaks(ecg):
    ecg_auto_peaks = autopeaks.AutoPeaks(thres=0.70,min_dist=300,fs=500)
    b = np.loadtxt("filter/ecg_notch_b.csv",delimiter=",")
    a = np.loadtxt("filter/ecg_notch_a.csv",delimiter=",")
    ecg_notch = lfilter(b,a,ecg)
    list(map(ecg_auto_peaks.findpeaks, ecg_notch))
    ecg_peak_indices = ecg_auto_peaks.peak_indexes
    ecg_peak_values = ecg_auto_peaks.peak_values
    ecg_rr_psg = np.diff(ecg_peak_indices)
    return ecg_rr_psg,ecg_peak_indices,ecg_peak_values,ecg_notch

In [126]:
hwm_rr, hwm_peak_indices,hwm_peak_values,hwm_notch = ecg_findpeaks(hwm_ecg)

In [129]:
plot([go.Scatter(x=hwm_peak_indices[1:], y=(30000/hwm_rr), mode="lines+markers"), go.Scatter(y=0.01*np.asarray(hwm_ecg))])

'file:///home/guo/Github/BCGHeart/bcgpeaks/temp-plot.html'

In [145]:
plot([go.Scatter(x=hwm_peak_indices[1:], y=(30000/hwm_rr), mode="lines+markers")])

'file:///home/guo/Github/BCGHeart/bcgpeaks/temp-plot.html'

# A001_1811280035 analysis

In [131]:
A035_path = "/home/guo/data/psg/physio/A001_1811280035.edf"

In [132]:
A035_ecg = load_ecg(A035_path)

In [143]:
A035_rr,A035_peak_indices,A035_peak_values,A035_notch = ecg_findpeaks(A035_ecg)

In [144]:
plot([go.Scatter(x=A035_peak_indices[1:], y=(30000/A035_rr), mode="lines+markers")])

'file:///home/guo/Github/BCGHeart/bcgpeaks/temp-plot.html'

In [135]:
plot([go.Scatter(x=A035_peak_indices[1:], y=(30000/A035_rr), mode="lines+markers"), go.Scatter(y=0.01*np.asarray(A035_ecg))])

'file:///home/guo/Github/BCGHeart/bcgpeaks/temp-plot.html'

In [141]:
plot([go.Scatter(y=np.asarray(A035_ecg))])

'file:///home/guo/Github/BCGHeart/bcgpeaks/temp-plot.html'

# psg_test

In [147]:
test_path = "/home/guo/data/psg/physio/psg_test.edf"

In [164]:
test_path = "/home/guo/data/psg/physio/A001_1812050006.edf"

In [165]:
test_ecg = load_ecg(test_path)

In [166]:
test_rr,test_peak_indices,test_peak_values,test_notch = ecg_findpeaks(test_ecg)

In [168]:
plot([go.Scatter(y=30000/test_rr,mode="lines+markers")])

'file:///home/guo/Github/BCGHeart/bcgpeaks/temp-plot.html'

In [162]:
plot([go.Scatter(y=test_notch[:100000],mode="lines")])

'file:///home/guo/Github/BCGHeart/bcgpeaks/temp-plot.html'

In [ ]:
# 